In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from urllib.request import urlretrieve
from owslib.wfs import WebFeatureService
from dotenv import load_dotenv
from pyspark.sql import functions as F

# Create a spark session
spark = (
    SparkSession.builder.appName("analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "6g")
    .getOrCreate()
)

22/09/30 19:29:00 WARN Utils: Your hostname, AryansLaptop resolves to a loopback address: 127.0.1.1; using 172.25.9.137 instead (on interface eth0)
22/09/30 19:29:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/30 19:29:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Transactions per merchant per day

In [2]:
finaldf = spark.read.parquet("../data/curated/finaldf.parquet/")
finaldf.limit(5)

order_datetime,merchant_abn,user_id,postcode,consumer_id,dollar_value,order_id,state,gender,company_name,tags,take_rate,revenue_band,total_pop,under10_pop,adolsc_pop,yng_adult_pop,mid_age_pop,old_pop,num_earners,median_age,median_income,mean_income,consumer_fraud_%,merchant_fraud_%
2021-08-21,94729574738,16982,4821,23684,156.3091245999424,407ce815-54e1-4ae...,QLD,Female,Scelerisque Corpo...,computer programm...,4.51,b,3128,468,271,707,947,735,1972,44,45102,50599,0.1,0.1
2022-04-24,45572698303,16982,4821,23684,115.38672513864692,72160f0d-06c5-432...,QLD,Female,Libero Proin Corp.,shoe shops,3.30,b,3128,468,271,707,947,735,1972,44,45102,50599,0.1,0.1
2021-08-21,34695415993,16982,4821,23684,35.730567249104645,d4524ef6-1f75-4f2...,QLD,Female,Ultrices Vivamus ...,"cable, satellite,...",6.02,a,3128,468,271,707,947,735,1972,44,45102,50599,0.1,0.1
2022-04-24,67400260923,16982,4821,23684,73.85811747094266,7acb9a9f-3ef6-4d4...,QLD,Female,Eleifend PC,computer programm...,5.97,a,3128,468,271,707,947,735,1972,44,45102,50599,0.1,0.1
2021-08-19,86578477987,16982,4821,23684,41.45540307953735,e8be544a-a311-432...,QLD,Female,Leo In Consulting,"watch, clock, and...",6.43,a,3128,468,271,707,947,735,1972,44,45102,50599,0.1,0.1


In [3]:
from pyspark.sql.functions import *

modeldf = finaldf.withColumn("year", date_format(col("order_datetime"), "yyyy").cast("long"))
modeldf = modeldf.withColumn("month", date_format(col("order_datetime"), "MM").cast("long"))
modeldf = modeldf.withColumn("day", date_format(col("order_datetime"), "dd").cast("long"))

for field in ("postcode","take_rate"):
    modeldf = modeldf.withColumn(
        field,
        col(field).cast('double')
    )

modeldf = modeldf.drop("order_datetime","order_id","company_name","consumer_fraud_%","merchant_fraud_%")

modeldf.limit(5)

merchant_abn,user_id,postcode,consumer_id,dollar_value,state,gender,tags,take_rate,revenue_band,total_pop,under10_pop,adolsc_pop,yng_adult_pop,mid_age_pop,old_pop,num_earners,median_age,median_income,mean_income,year,month,day
94729574738,16982,4821.0,23684,156.3091245999424,QLD,Female,computer programm...,4.51,b,3128,468,271,707,947,735,1972,44,45102,50599,2021,8,21
45572698303,16982,4821.0,23684,115.38672513864692,QLD,Female,shoe shops,3.3,b,3128,468,271,707,947,735,1972,44,45102,50599,2022,4,24
34695415993,16982,4821.0,23684,35.730567249104645,QLD,Female,"cable, satellite,...",6.02,a,3128,468,271,707,947,735,1972,44,45102,50599,2021,8,21
67400260923,16982,4821.0,23684,73.85811747094266,QLD,Female,computer programm...,5.97,a,3128,468,271,707,947,735,1972,44,45102,50599,2022,4,24
86578477987,16982,4821.0,23684,41.45540307953735,QLD,Female,"watch, clock, and...",6.43,a,3128,468,271,707,947,735,1972,44,45102,50599,2021,8,19


In [4]:
modeldf = modeldf.groupBy("year","month","day","merchant_abn").agg(
    count("dollar_value").alias("daily_transactions"),
)

modeldf.limit(5)

year,month,day,merchant_abn,daily_transactions
2021,3,2,45899477665,7
2022,6,24,60978195146,45
2021,9,27,55179323922,5
2022,2,13,71041015148,20
2021,7,3,46804135891,281


In [5]:
modeldf.write.mode("overwrite").parquet("../data/curated/transmodeldf.parquet")

22/09/30 19:29:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers
22/09/30 19:29:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/09/30 19:29:33 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 84.45% for 16 writers
22/09/30 19:29:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 90.08% for 15 writers
22/09/30 19:29:34 WARN MemoryManager: Total allocation exceeds 95.00% (1,813,485,955 bytes) of heap memory
Scaling row group sizes to 96.51% for 14 writers


In [5]:
from scipy.stats import f_oneway

df = modeldf.toPandas()

print(f_oneway(df["year"],df["daily_transactions"]))
print(f_oneway(df["month"],df["daily_transactions"]))
print(f_oneway(df["day"],df["daily_transactions"]))
print(f_oneway(df["merchant_abn"],df["daily_transactions"]))

F_onewayResult(statistic=5926048737.94882, pvalue=0.0)
F_onewayResult(statistic=14235.494160709548, pvalue=0.0)
F_onewayResult(statistic=48306.67730233565, pvalue=0.0)
F_onewayResult(statistic=5292918.128755565, pvalue=0.0)


In [6]:
sdf, tdf = modeldf.randomSplit([0.9, 0.1], seed=22)
print(sdf.count(),tdf.count())
sdf.limit(5)

1086970 120611


year,month,day,merchant_abn,daily_transactions
2021,2,28,11802512399,1
2021,2,28,12497320096,1
2021,2,28,13118172970,13
2021,2,28,14602793938,1
2021,2,28,15061716544,21


In [7]:
# VectorAssembler creates new vectors from existing columns
from pyspark.ml.feature import VectorAssembler

features = 'features'
input_cols = ["year","month","day","merchant_abn"]

assembler = VectorAssembler(
    # which column to combine
    inputCols=input_cols, 
    # How should the combined columns be named
    outputCol=features
)

model_sdf = assembler.transform(sdf.dropna('any'))
# Display the features and targets for our model
model_sdf.select('features').head(5), model_sdf.select('daily_transactions').head(5)

([Row(features=DenseVector([2021.0, 2.0, 28.0, 11802512399.0])),
  Row(features=DenseVector([2021.0, 2.0, 28.0, 12497320096.0])),
  Row(features=DenseVector([2021.0, 2.0, 28.0, 13118172970.0])),
  Row(features=DenseVector([2021.0, 2.0, 28.0, 14602793938.0])),
  Row(features=DenseVector([2021.0, 2.0, 28.0, 15061716544.0]))],
 [Row(daily_transactions=1),
  Row(daily_transactions=1),
  Row(daily_transactions=13),
  Row(daily_transactions=1),
  Row(daily_transactions=21)])

In [8]:
from pyspark.ml.regression import DecisionTreeRegressor

model = DecisionTreeRegressor(
    featuresCol='features', 
    labelCol='daily_transactions',
    maxDepth=12,
    maxBins=1000
).fit(model_sdf)

In [9]:
# preprocess for predictions
predict_test = tdf.select(*input_cols)

assembler = VectorAssembler(
    inputCols=input_cols, 
    outputCol=features
)

predict_sdf = assembler.transform(predict_test).select(features)

#predict_sdf.show(1, vertical=True)

In [10]:
predictions = model.transform(predict_sdf)
#predictions.limit(5)

In [11]:
import numpy as np
ytest =  np.array(tdf.select("daily_transactions").collect())
ypred = np.array(predictions.select("prediction").collect())
#ypred

In [12]:
import numpy as np
from sklearn.metrics import mean_absolute_error as mae
print(np.round(mae(ytest,ypred), 2))
print(ytest.min(),ytest.max(),ytest.mean(),ytest.std())

9.08
1 816 9.766198771256352 28.575545312022527


In [13]:
print(tdf.groupBy("merchant_abn","month").count().orderBy("count", ascending = False))

+------------+-----+-----+
|merchant_abn|month|count|
+------------+-----+-----+
| 22487474791|    4|   16|
| 52535771754|    5|   16|
| 49505931725|    5|   16|
| 69703285964|    9|   15|
| 71236652596|    4|   15|
| 25732256442|    9|   15|
| 66678764219|    3|   15|
| 21359184622|    7|   15|
| 59162205130|    3|   15|
| 97738870301|    8|   15|
| 41271931352|    8|   14|
| 82076270246|    9|   14|
| 80753862231|    8|   14|
| 73343245974|    6|   14|
| 21025433654|   10|   14|
| 99938978285|    6|   14|
| 61481587663|    4|   14|
| 31585975447|    9|   14|
| 43660707274|    3|   14|
| 32844138686|    5|   14|
+------------+-----+-----+
only showing top 20 rows



In [14]:
tdf.limit(5)

year,month,day,merchant_abn,daily_transactions
2021,2,28,14639241847,3
2021,2,28,22614311069,1
2021,2,28,35318110910,3
2021,2,28,37243721612,2
2021,2,28,52529569808,1
